In [12]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import pandas as pd

# Initialize the LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model="llama-3.3-70b-versatile"
)

# Load and extract job posting data from the Amazon job page
loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2828235/software-development-engineer-2025-us")
page_data = loader.load().pop().page_content

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job posting and return it in JSON format containing the 
    following keys: `company name`, `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):    
    """
)

chain_extract = prompt_extract | llm 
res_extract = chain_extract.invoke(input={'page_data': page_data})

json_parser = JsonOutputParser()
job = json_parser.parse(res_extract.content)

# Load roles.csv that has columns: Role, Techstack, Links
df = pd.read_csv("roles.csv")

# Compare job title (from extracted JSON) with the 'Role' column in roles.csv
job_title = job.get('role', '')
if job_title:
    # Perform a case-insensitive search to match the job title with roles in the CSV
    matched_rows = df[df['Role'].str.lower().str.contains(job_title.lower())]
    if not matched_rows.empty:
        matched_techstack = ", ".join(matched_rows['Techstack'].unique())
    else:
        matched_techstack = "No matching tech stack skills found based on the job title"
else:
    matched_techstack = "No job title extracted from the job posting"

# Prepare a prompt template to generate a cold email only, using additional context from the CSV comparison.
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### ADDITIONAL INFORMATION:
    Based on my portfolio (roles.csv), the tech stack skills that match the job title are: {matched_techstack}.
    This comparison underscores my proficiency in areas directly relevant to the position.
    
    ### INSTRUCTION:
    You are Divya Dadi. I recently graduated from Rice University with a Master’s in Computer Science and have robust experience in both Software Engineering and Data Analytics. As a Software Engineer at Affekta LLC, I developed and maintained data transformation pipelines that improved course import efficiency by 40% for an AI-driven adaptive learning platform, and I structured and validated JSON-based data models to enhance content accuracy. During my internship at American Unit Inc, I supported the development of a Privacy Data Management System by optimizing SQL queries and designing efficient ETL pipelines. At Tata Consultancy Services, I designed and deployed interactive Power BI dashboards, improving reporting accuracy by 25% and enabling data-driven decision-making.
    
    Your task is to write a cold email to the hiring manager regarding the above job posting. The cold email should highlight how my skills and the matched tech stack from my portfolio align perfectly with the job requirements.
    Do not include any additional links apart from my LinkedIn profile.
    Please include my LinkedIn profile: https://www.linkedin.com/in/divya-dadi-9a2539172/
    
    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm
res_email = chain_email.invoke({
    "job_description": str(job),
    "matched_techstack": matched_techstack
})

print(res_email.content)


Subject: Application for Software Development Engineer at Amazon

Dear Hiring Manager,

I am excited to apply for the Software Development Engineer position at Amazon, as advertised. With a Master's in Computer Science from Rice University and a strong background in Software Engineering and Data Analytics, I am confident that my skills align perfectly with the job requirements.

Although my portfolio did not yield a direct match with the job title, my experience and skills in software development, data transformation, and data analytics make me an ideal candidate for this role. My proficiency in programming languages, data modeling, and data management will enable me to create new products, features, and services from scratch, while managing ambiguity and pace.

As a Software Engineer at Affekta LLC, I developed and maintained data transformation pipelines, improving course import efficiency by 40% for an AI-driven adaptive learning platform. I also structured and validated JSON-based 